In [ ]:
import sys
sys.path.append('../')
sys.path.append('../tests/python-support/')

%matplotlib notebook

In [ ]:
import matplotlib
import experiments
import itertools
import util

In [ ]:
# Set up details of experiments
final_time = [1.0]
sample_count = [ 10001 ]
h = [ 0.5 ** x for x in [ 3, 3.5, 4 ] ]

param_collection = itertools.product(final_time, sample_count, h)
params = [ { 'T': str(x[0]), 'N': str(x[1]), 'h': str(x[2]) } for x in param_collection]

In [ ]:
# Run experiments and collect results
results = experiments.run_experiments(params)

In [ ]:
num_dof = [ result['mesh']['num_vertices'] for result in results ]
l2_error = [ result['error_summary']['l2'] for result in results ]
h1_error = [ result['error_summary']['h1'] for result in results ]

In [ ]:
util.estimate_slope(num_dof, l2_error)

In [ ]:
util.estimate_slope(num_dof, h1_error)